# We have database of one of the services for book's app. It contains data on books, publishers, authors, and customer ratings and reviews of books. We will analize book rating, publishers, rating of authors and amount of reviews.

# This information will be used to create a value proposition for the new product.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('max_colwidth', 400)
#! pip install psycopg2

In [2]:
#Lets establish connection to the database
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Study the tables (print the first rows).


In [3]:
tables= ['books','reviews','ratings','authors']

for i in tables:
    query= f'''SELECT * FROM {i} LIMIT 5'''
    display(pd.io.sql.read_sql(query, con = engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


# Find the number of books released after January 1, 2000.


In [4]:
query2='''SELECT count (publication_date)
FROM books
 '''
example_df=pd.io.sql.read_sql(query2, con = engine)
display(example_df)

,count
0,1000


In [5]:
query1='''SELECT count (publication_date)
FROM books
WHERE CAST(publication_date AS date)>= '2000-01-01'
 '''
example_df=pd.io.sql.read_sql(query1, con = engine)
display(example_df)

,count
0,821


821 book released after january 1,2020. The total number of books is 1000, so in 2000 year and after released more then 80% of books here.

# Find the number of user reviews and the average rating for each book.


In [22]:

query3="""SELECT rt.book_id, avg(DISTINCT rt.rating),count(DISTINCT rv.review_id)
FROM reviews AS rv
RIGHT JOIN ratings AS rt ON rt.book_id=rv.book_id
GROUP BY rt.book_id
;
"""
example_df=pd.io.sql.read_sql(query3, con = engine)
display(example_df)

,book_id,avg,count
0,1,3.666667,2
1,2,2.500000,1
2,3,4.500000,3
3,4,4.500000,2
4,5,4.000000,4
...,...,...,...
995,996,4.000000,3
996,997,3.666667,3
997,998,3.333333,4
998,999,4.500000,2


# Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [7]:
query5="""SELECT count(book_id) AS amount_of_books, publisher_id
FROM books 
WHERE num_pages > 50
GROUP BY publisher_id
ORDER BY count(book_id) DESC
LIMIT 1;
"""
example_df=pd.io.sql.read_sql(query5, con = engine)
display(example_df)

,amount_of_books,publisher_id
0,42,212


The publisher 212 has released the greatest number of books more than 50 pages and it's amount 42.

# Identify the author with the highest average book rating: look only at books with at least 50 ratings.

In [8]:
query8="""SELECT  a.author,
avg(r.rating)

FROM author AS a
JOIN books AS b ON a.author_id=b.author_id
JOIN ratings AS r ON r.book_id=b.book_id
GROUP BY a.author
HAVING count(r.rating) > 50
ORDER BY avg(r.rating) DESC
LIMIT 1;

"""
example_df=pd.io.sql.read_sql(query8, con = engine)
display(example_df)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.288462


J.K. Rowling/Mary GrandPré has the  highest average book rating.

# Find the average number of text reviews among users who rated more than 50 books.

In [20]:
query10="""SELECT AVG(t.reviews)
FROM (SELECT count(DISTINCT rv.review_id) AS reviews
FROM ratings as rt
JOIN reviews AS rv ON rv.username=rt.username
GROUP BY rt.username
HAVING COUNT(DISTINCT rt.rating_id) > 50) as t

"""
example_df=pd.io.sql.read_sql(query10, con = engine)
display(example_df)

,avg
0,24.333333


24 is average number of text reviews among users who rated more than 50 books 

821 book released after january 1,2020. 

The total number of books is 1000, so in 2000 year and after released more then 80% of books here.

The publisher 212 has released the greatest number of books more than 50 pages and it's amount 42. So it's about 4% from whole books that we have.

J.K. Rowling/Mary GrandPré has the  highest average book rating and it's around 4.3. 

24 is average number of text reviews among users who rated more than 50 books, this is quite a large number of reviews from users if we talking about 1000books at all.